<a href="https://colab.research.google.com/github/YanBoChen0928/YanBoChenPortfolio-Github/blob/main/2023_08_27_Blackjack_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random, sys

#set up:
HEARTS = chr(9829)
DIAMONDS = chr(9830)
SPADES = chr(9824)
CLUBS = chr(9827)
BACKSIDE = 'backside'


def main():
  print('''

  Rules:
    Try to get as close to 21 without going over.
    Kings, Queens, and Jacks are worth 10 points.
    Aces are worth 1 or 11 points.
    Cards 2 through 10 are worth their face value.
    (H)it to take another card.
    (S)tand to stop taking cards.
    On your first play, you can (D)ouble down to increase your bet
    but must hit exactly one more time before standing.
    In case of a tie, the bet is returned to the player.
    The dealer stops hitting at 17.''')

  money = 3000
  while True:
    # Check if the player has run out of money:
    if money <= 0:
      print("You're brankrupt!")
      print("Good thing you weren't playing with real money.")
      print("Thanks for playing!")
      sys.exit()

    # Let the player enter their bet:
    print("Money:", money)
    bet = getBet(money)

    # Give the dealer and player two cards from the deck per person:
    deck = getDeck()
    dealerHand = [deck.pop(), deck.pop()] # randomly draw two cards to dealer. because of pop(), it will delete the card it drew.
    playerHand = [deck.pop(), deck.pop()]

    # 主loop: initially Handle player actions:
    print("Bet:", bet)
    while True:
      displayHands(playerHand, dealerHand, False)
      print()
      # check if the player has bust: no money decrease??
      if getHandValue(playerHand) > 21:
        break
      # get the player's move, either H,S, or D:
      move = getMove(playerHand, money - bet)
      # handle the player actions:
      if move == 'D': # additional bet
        additionalBet = getBet(min(bet,(money - bet)))
        bet += additionalBet
        print('Bet increased to {}.'.format(bet))
        print('Bet:', bet)

      if move in ('H','D'):
        newCard = deck.pop()
        rank, suit = newCard
        print('You drew a {} of {}.'.format(rank,suit))
        playerHand.append(newCard)

        if getHandValue(playerHand) > 21:
          # The player has busted:
          continue # 回到最上面loop--> 因爲 playHand > 21, 會跳出主loop

      if move in ('S','D'):
        break

      # handle the dealer's actions:
      if getHandValue(playerHand) <= 21:
       while getHandValue(dealerHand) < 17: #規則：專家少於17點的時候會hit牌
          # The dealer hits:
          print('Dealer hits...')
          dealerHand.append(deck.pop())
          displayHands(playerHand, dealerHand, False)

          if getHandValue(dealerHand) > 21:
            break # The dealer has busted.
          input('Press Enter to continue...')
          print('\n\n')

    # show the final hands:
    displayHands(playerHand, dealerHand, True)

    playerValue = getHandValue(playerHand)
    dealerValue = getHandValue(dealerHand)

    # 走完上面發牌loop後來判斷勝負：handle whether the player won, lost, or tied:
    if dealerValue > 21:
        print('Dealer bust! You win ${}!'.format(bet))
        money += bet
    elif (playerValue > 21) or (playerValue < dealerValue):
        print('You lost!')
        money -= bet
    elif playerValue > dealerValue:
        print('You won ${}!'.format(bet))
        money += bet
    elif playerValue == dealerValue:
        print('It\'s a tie, the bet is returned to you.')

    print('Now you have ${}'.format(money))
    user_input = input('Press Enter to continue...or QUIT to quit: ')
    if user_input.upper() == 'QUIT':
      sys.exit()  # 使用 sys.exit() 退出程序
    print('\n\n')


def getBet(maxBet):
  '''Ask the player how much they want to bet for this round.'''
  while True: #keep asking until they enter a valid amount.
    print(f'How much do you bet? (1-{maxBet}, or QUIT)')
    bet = input('> ').upper().strip()                                              # try to add int() 2023.08.26 but it's unnecessary
    if bet == 'QUIT':
      print('Thanks for playing!')
      sys.exit()

    if not bet.isdecimal():
      continue # if the player didn't enter a number, ask again. (false), if true, 下一步

    bet = int(bet)
    if 1 <= bet <= maxBet:
      return bet #player entered a valid bet


def getDeck():
  '''Return a list of (rank, suit)tuples for all 52 cards.'''
  deck = []
  for suit in (HEARTS, DIAMONDS, SPADES, CLUBS):
    for rank in range(2,11):
      deck.append((str(rank),suit))
    for rank in ('J','Q','K','A'):
      deck.append((rank,suit))
  random.shuffle(deck) #shuffle只可以用在list或可以變的列表
  return deck


def getHandValue(cards): #cards main() how to define??
  '''Returns the value of the cards. Face cards are worth 10, aces are worth 11 or 1
  (this function picks the most suitable ace value).'''
  value = 0
  numberOfAces = 0

  #Add the value for the non-ace cards:
  for card in cards:
    rank = card[0] # card is a tuple like (rank, suit)
    if rank == 'A':
      numberOfAces += 1
    elif rank in ('K','Q','J'): # Face cards are worth 10 points.
      value += 10
    else:
      value += int(rank) # Numbered cards are worth their number.

  # Add the value for Aces:
  value += numberOfAces # Add 1 per ace.
  for i in range(numberOfAces):
    #if another 10 can be added with busting, do so:
    if value + 10 <=21:
      value += 10
  return value


def displayCards(cards):
  '''Display all the cards in the cards list.'''
  rows = ['', '', '', '', ''] #the text to display on each row.

  for i, card in enumerate(cards):
    rows[0] += ' ___  '  #print the top line of the card.
    if card == BACKSIDE:
      # print a card's back:
      rows[1] += '|## | '
      rows[2] += '|###| '
      rows[3] += '| ##| '
    else:
      # print the card's front:
      rank, suit = card #the card is a tuple data structure.
      rows[1] += '|{} | '.format(rank.ljust(2))
      rows[2] += '| {} | '.format(suit)
      rows[3] += '|_{}| '.format(rank.rjust(2, '_'))

  #print each row on the screen:
  for row in rows:
    print(row)

def displayHands(playerHand, dealerHand, showDealerHand):
  '''Show the player's and dealer's cards. Hide the dealer's first card if showDealerHand is False.'''
  print()
  if showDealerHand:
    print('Dealer:', getHandValue(dealerHand))
    displayCards(dealerHand)
  else:
    print('Dealer: ???')
    #Hide the dealer's first card:
    displayCards([BACKSIDE] + dealerHand[1:])

  #show the player's cards:
  print('Player:', getHandValue(playerHand))
  displayCards(playerHand)

def getMove(playerHand, money):
  '''Asks the player for their move, and return 'H' for hit, 'S' for stand, and 'D' for double down.'''
  while True: #keep looping until the player enters a correct move
    #determine what moves the player can make:
    moves = ['(H)it','(S)tand']

    #The player can double down on their first move, which we can tell because they'll have exactly two cards:
    if len(playerHand) == 2 and money > 0:
      moves.append('(D)ouble down')

    #get the player's move:
    movePrompt =', '.join(moves) + '> ' # will display: (H)it, (S)tand, (D)ouble down.
    move = input(movePrompt).upper()
    if move in ('H','S'):
      return move
    if move == 'D' and '(D)ouble down' in moves: #ensure that D is valid value
      return move

if __name__ == '__main__':
  main()

if __name__ == "__main__" 的意思是：如果這個 python 模組被當作主要的程式執行，而非被 import 成為其他模組的一部分，那麼以下的程式碼才會被執行。

也就是說，如果這個模組被當作主程式執行，那麼 if __name__ == "__main__" 後面的程式碼會被執行；但如果這個模組被 import 成為其他模組的一部分，那麼 if __name__ == "__main__" 後面的程式碼就不會被執行。

所以當我們把 if __name__ == "__main__" 放在一個程式碼檔案的最後面時，即使沒有定義 main() 函式，只要這個檔案被當作主程式執行，它裡面的所有程式碼都會被執行。

In [ ]:
def my_function():
    """
    This is a docstring for my_function.
    It can contain multiple lines of text.
    """
print(my_function())

None
